In [1]:
import os
import pandas as pd
from tqdm import tqdm
import dashscope

INPUT_FILE  = "合理用药数据集v4.1-测试用.xlsx"
OUTPUT_FILE = "通义千问_qwen-plus结果_new.xlsx"

dashscope.api_key = os.getenv("DASHSCOPE_API_KEY", "") #这里是key位置

# 各工作表对应的提示词
PROMPTS = {
    "单选": "以下是关于中药处方审核的单选题，请根据规则选择正确的选项。仅输出选项即可。",
    "多选": "以下是关于中药处方审核的多选题，请根据规则选择所有正确的选项。仅输出选项即可。"
}

# ========== 通用调用函数 ==========
def call_qwen(prompt: str, question: str, candidate_answers: str) -> str:
    try:
        response = dashscope.Generation.call(
            model="qwen-plus",  
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"题目：{question}\n选项：{candidate_answers}"}
            ],
            result_format="message", 
            max_tokens=8,
            temperature=0
        )
        ans = response["output"]["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print("API Error:", e)
        ans = "Error"

    # 仅保留 A-Z、分隔符
    ans = "".join(c for c in ans if c in "ABCDEFGHIJKLMNOPQRSTUVWXYZ、，, ")
    ans = ans.replace("，", "、").replace(",", "、").strip()
    return ans

# ========== 主流程 ==========
xls = pd.ExcelFile(INPUT_FILE)

with pd.ExcelWriter(OUTPUT_FILE) as writer:
    for sheet in ["单选", "多选"]:
        if sheet not in xls.sheet_names:
            print(f"⚠️ 找不到工作表：{sheet}，跳过")
            continue

        df = pd.read_excel(xls, sheet_name=sheet)
        prompt = PROMPTS[sheet]

        model_answers = []
        for _, row in tqdm(df.iterrows(), desc=f"{sheet} 处理中"):
            question = row["Question"]
            candidate_answers = row["Candidate answers"]
            ans = call_qwen(prompt, question, candidate_answers)
            model_answers.append(ans)

        df["模型答案"] = model_answers
        df.to_excel(writer, sheet_name=sheet, index=False)

print("✅ 处理完成，结果已保存到：", OUTPUT_FILE)

单选 处理中: 36it [00:37,  1.06it/s]

API Error: 'NoneType' object is not subscriptable


单选 处理中: 297it [05:40,  1.15s/it]
多选 处理中: 31it [00:34,  1.10s/it]

✅ 处理完成，结果已保存到： 通义千问_qwen-plus结果_new.xlsx


In [2]:
import pandas as pd
import re

INPUT_FILE = "通义千问_qwen-plus结果_new.xlsx"
OUTPUT_FILE = "通义千问_qwen-plus结果_new_清洗后.xlsx"

def clean_answer(ans):
    """清洗模型答案"""
    if pd.isna(ans):
        return ""
    # 提取所有大写字母
    letters = re.findall(r"[A-Z]", str(ans).upper())
    if len(letters) == 1:
        return letters[0]        
    return " ".join(letters)      

# 读取
xls = pd.ExcelFile(INPUT_FILE)

with pd.ExcelWriter(OUTPUT_FILE) as writer:
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        if "模型答案" in df.columns:
            df["模型答案"] = df["模型答案"].apply(clean_answer)
        df.to_excel(writer, sheet_name=sheet, index=False)

print("清洗完成 →", OUTPUT_FILE)

清洗完成 → 通义千问_qwen-plus结果_new_清洗后.xlsx
